In [6]:
from docuverse.engines import SearchResult
from docuverse.engines import SearchEngine
from docuverse.engines.search_engine_config_params import DocUVerseConfig
from docuverse.utils import get_param
import pickle

In [8]:
config="experiments/zdocs/milvus_dense.granite-125m.test.yaml"
engine=SearchEngine(DocUVerseConfig(config))

Retrieval engine: milvus-dense
Running on the gpus:  ['NVIDIA GeForce RTX 3090', 'NVIDIA GeForce RTX 3090']
=== done initializing model


In [14]:
results=engine.read_output()

In [16]:
res=results[0]

In [17]:
len(res)

35

In [71]:
urls=[[get_param(d, 'metadata.url') for d in res] for res in results]
qurls = [d.question.get('metadata')['gold-urls'] for d in results]
qtext = [d.question.text for d in results]

In [47]:
data=engine.read_data()

Cache filename is /home/raduf/.local/share/elastic_ingestion/benchmark__zdocs__corpus.jsonl.bz2_512_100_True_all_granite-embedding-125m-english.pickle.xz
Skipped 0 passages.


Computing statistics: 100%|██████████| 1167439/1167439 [00:02<00:00, 443468.68it/s]


Statistics:
Number of documents:1026632   
Number of tiles:    1167439   
#tiles per document:1.14      
                    Character-based:    Token-based:        
  Minimum length:   9                   6                   
  Maximum length:   2240                512                 
  Average length:   1084.2              269.6               
Char histogram:

# NumSamples = 1167439; Min = 9.00; Max = 2240.00
# Mean = 1084.212026; Variance = 406224.673368; SD = 637.357571; Median 977.000000
# each ∎ represents a count of 2632
    9.0000 -   232.1000 [ 55241]: ∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎
  232.1000 -   455.2000 [197448]: ∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎
  455.2000 -   678.3000 [153778]: ∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎
  678.3000 -   901.4000 [133010]: ∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎
  901.4000 -  1124.5000 [128056]: ∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎∎
 1124.5000 -  1347.6000 [ 91542]: ∎∎∎∎∎

In [52]:
url2id = {}
for d in data:
    url2id[d['metadata']['url']] = d['id']

In [80]:
from rouge_score.rouge_scorer import RougeScorer
rouge_scorer = RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
def compute_rouge_matches(result, thr=0.9):
    gold = result.question.get('metadata.answer')

    scores = []
    for answer in result:
        score = rouge_scorer.score(gold, answer.text)
        scores.append(score['rouge1'].recall)

    inds = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)
    # print(f"Best answer for {res.question.text}: {res[inds[0]]['id']}, recall: {scores[inds[0]]}")
    # return {result.question.id: {"best": result[inds[0]]['id'], "score": scores[inds[0]]}}
    res = []
    for ansid in inds:
        if scores[ansid] >= thr:
            res.append({'id': result[ansid]["id"], 'score': scores[ansid]})
    return res

In [83]:
compute_rouge_matches(res, 0.9)

[{'id': '451813522832071401-0-1261', 'score': 1.0}]

In [86]:
compute_rouge_matches(results[3])

[{'id': '451813522833601023-0-823', 'score': 1.0}]

In [90]:
import json

questions = [json.loads(s) for s in open("../benchmark/zdocs/questions_all.jsonl").readlines()]

In [100]:
from tqdm.notebook import tqdm
from docuverse.utils import get_orig_docid
outq = questions.copy()
for i, q in tqdm(enumerate(outq)):
    matches = compute_rouge_matches(results[i])
    if len(matches) > 0:
        q['metadata']['relevant'] = [get_orig_docid(m['id']) for m in matches]
        q['metadata']['rouge_scores'] = [m['score'] for m in matches]
    else:
        q['metadata']['rouge_scores'] = []
        q['metadata']['relevant'] = [url2id[q['metadata']['gold-urls']]] if q['metadata']['gold-urls'] in url2id else []

0it [00:00, ?it/s]

In [101]:
outq[199]

{'metadata': {'gold-urls': 'https://www.ibm.com/docs/en/ztpf/2023?topic=zbxxx-zbevf-set-set-business-event-processing-control-values',
  'answer': 'Use the ZBEVF SET command with the MAXECBs parameter.',
  'fname': '/Users/vineet/data/systemz/data/predicted_ans_with_analysis_20250121/tpf_gds_2023-09-23.res.csv',
  'rouge_scores': [],
  'relevant': []},
 '_id': 'tpf_gds_77',
 'text': 'How do I set the number of concurrent dispatch ECBs for a dispatch queue?'}

In [129]:
open("../benchmark/zdocs/missing_urls.txt", "w").writelines([q['metadata']['gold-urls']+"\n" for q in outq if q['metadata']['relevant']==[] and q['metadata']['answer']!=""])

'../benchmark/zdocs/latest?topic=scenarios-overview-ims-queue-structures\n'